**Neural Network - All Machines with Date Included**

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd
import numpy as np
import tensorflow as tf
import datetime as dt

In [ ]:
# Import our input dataset
all_tele_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/Resources/all_tele_main.csv')
all_tele_df = all_tele_df.sort_values(by=['datetime']).drop_duplicates(subset=['datetime', 'machineid'], keep='first').reset_index(drop=True)
all_tele_df.head()

<ipython-input-2-51b3cfc196db>:2: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  all_tele_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/Resources/all_tele_main.csv')


,datetime,machineid,machinemodel,machineage,volt,rotate,pressure,vibration,errorid,compmaint,compfail
0,2015-01-01 06:00:00,34,model4,10,181.859394,483.215401,82.463735,37.849856,0,comp2,0
1,2015-01-01 06:00:00,41,model4,9,162.314459,424.540949,129.113722,52.591978,0,0,0
2,2015-01-01 06:00:00,39,model4,0,150.138449,421.747438,100.174935,40.367037,0,0,0
3,2015-01-01 06:00:00,10,model3,10,158.421261,500.830885,119.750673,45.571344,0,0,0
4,2015-01-01 06:00:00,45,model3,14,191.649670,383.200891,85.630577,52.169106,0,0,0


In [ ]:
all_tele_df.dtypes

datetime         object
machineid         int64
machinemodel     object
machineage        int64
volt            float64
rotate          float64
pressure        float64
vibration       float64
errorid          object
compmaint        object
compfail          int64
dtype: object

In [ ]:
# update date to integer ready for testing
all_tele_df['datetime'] = pd.to_datetime(all_tele_df['datetime'])
all_tele_df['datetime'] = all_tele_df['datetime'].astype(np.int64)
all_tele_df.dtypes


datetime          int64
machineid         int64
machinemodel     object
machineage        int64
volt            float64
rotate          float64
pressure        float64
vibration       float64
errorid          object
compmaint        object
compfail          int64
dtype: object

In [ ]:
# use get_dummies to convert categorical variables
all_tele_df = pd.get_dummies(data=all_tele_df, columns=['machinemodel', 'errorid', 'compmaint'])
all_tele_df

,datetime,machineid,machineage,volt,rotate,pressure,vibration,compfail,machinemodel_model1,machinemodel_model2,...,errorid_error2,errorid_error3,errorid_error4,errorid_error5,compmaint_0,compmaint_0,compmaint_comp1,compmaint_comp2,compmaint_comp3,compmaint_comp4
0,1420092000000000000,34,10,181.859394,483.215401,82.463735,37.849856,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1420092000000000000,41,9,162.314459,424.540949,129.113722,52.591978,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1420092000000000000,39,0,150.138449,421.747438,100.174935,40.367037,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1420092000000000000,10,10,158.421261,500.830885,119.750673,45.571344,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1420092000000000000,45,14,191.649670,383.200891,85.630577,52.169106,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876095,1451628000000000000,59,17,167.677354,418.758488,102.669403,43.633930,0,0,0,...,0,0,0,0,0,1,0,0,0,0
876096,1451628000000000000,50,4,211.600641,397.565518,121.283439,46.420810,0,0,0,...,0,0,0,0,1,0,0,0,0,0
876097,1451628000000000000,38,15,140.426314,481.873972,89.826135,38.081556,0,0,0,...,0,0,0,0,0,1,0,0,0,0
876098,1451628000000000000,43,14,175.002673,471.962268,97.606839,39.252586,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
# split preprocessed data into feature and target arrays
y = all_tele_df.compfail.values
X = all_tele_df.drop(columns='compfail').values

# split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 35
hidden_nodes_layer2 = 15

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 35)                875       
                                                                 
 dense_1 (Dense)             (None, 15)                540       
                                                                 
 dense_2 (Dense)             (None, 1)                 16        
                                                                 
Total params: 1431 (5.59 KB)
Trainable params: 1431 (5.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
20534/20534 [==============================] - 54s 3ms/step - loss: 0.0032 - accuracy: 0.9991
Epoch 2/100
20534/20534 [==============================] - 36s 2ms/step - loss: 0.0015 - accuracy: 0.9994
Epoch 3/100
20534/20534 [==============================] - 38s 2ms/step - loss: 0.0014 - accuracy: 0.9995
Epoch 4/100
20534/20534 [==============================] - 36s 2ms/step - loss: 0.0013 - accuracy: 0.9995
Epoch 5/100
20534/20534 [==============================] - 37s 2ms/step - loss: 0.0013 - accuracy: 0.9995
Epoch 6/100
20534/20534 [==============================] - 36s 2ms/step - loss: 0.0013 - accuracy: 0.9995
Epoch 7/100
20534/20534 [==============================] - 37s 2ms/step - loss: 0.0013 - accuracy: 0.9995
Epoch 8/100
20534/20534 [==============================] - 36s 2ms/step - loss: 0.0013 - accuracy: 0.9995
Epoch 9/100
20534/20534 [==============================] - 37s 2ms/step - loss: 0.0012 - accuracy: 0.9995
Epoch 10/100
20534/20534 [====================

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

6845/6845 - 8s - loss: 0.0029 - accuracy: 0.9995 - 8s/epoch - 1ms/step
Loss: 0.0029238767456263304, Accuracy: 0.9994886517524719


In [ ]:
# Making predictions using the testing data
predictions = nn.predict(X_test_scaled)
predictions_bool = np.argmax(predictions, axis=1)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions_bool)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions_bool)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions_bool))

6845/6845 [==============================] - 8s 1ms/step
Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,218845,0
Actual 1,180,0


Accuracy Score : 0.9991781760073051
Classification Report


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    218845
           1       0.00      0.00      0.00       180

    accuracy                           1.00    219025
   macro avg       0.50      0.50      0.50    219025
weighted avg       1.00      1.00      1.00    219025



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
